In [1]:
import tensorflow as tf
import cv2
import sys
sys.path.append("Wrapped Game Code/")
import pong_fun as game # whichever is imported "as game" will be used
import dummy_game
#import tetris_fun as game
import random
import time 
import numpy as np
from collections import deque

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
GAME = 'pong' # the name of the game being played for log files
ACTIONS = 6 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON = 0.05 # final value of epsilon
INITIAL_EPSILON = 1.0 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
K = 1 # only select an action every Kth frame, repeat prev for others

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def createNetwork():
    # network weights
    W_conv1 = weight_variable([8, 8, 4, 32])
    b_conv1 = bias_variable([32])

    W_conv2 = weight_variable([4, 4, 32, 64])
    b_conv2 = bias_variable([64])

    W_conv3 = weight_variable([3, 3, 64, 64])
    b_conv3 = bias_variable([64])
    
    W_fc1 = weight_variable([1600, 512])
    b_fc1 = bias_variable([512])

    W_fc2 = weight_variable([512, ACTIONS])
    b_fc2 = bias_variable([ACTIONS])

    # input layer
    s = tf.placeholder("float", [None, 80, 80, 4])

    # hidden layers
    h_conv1 = tf.nn.relu(conv2d(s, W_conv1, 4) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 2) + b_conv2)
    #h_pool2 = max_pool_2x2(h_conv2)

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 1) + b_conv3)
    #h_pool3 = max_pool_2x2(h_conv3)

    #h_pool3_flat = tf.reshape(h_pool3, [-1, 256])
    h_conv3_flat = tf.reshape(h_conv3, [-1, 1600])

    h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)

    # readout layer
    readout = tf.matmul(h_fc1, W_fc2) + b_fc2

    return s, readout, h_fc1

In [3]:
def trainNetwork(s, readout, h_fc1, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # open up a game state to communicate with emulator
    game_state = game.GameState()
    # past 3 wins 
    win_score = []
    win_score.append(0)
    win_score.append(0)
    win_score.append(0)
    win_score.append(0)
    # store the previous observations in replay memory
    D = deque()

    # printing
#     a_file = open("logs_" + GAME + "/readout.txt", 'w')
#     h_file = open("logs_" + GAME + "/hidden.txt", 'w')

    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal, bar1_score, bar2_score = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis = 2)

    # saving and loading networks
    saver = tf.train.Saver()
    sess.run(tf.initialize_all_variables())
#     checkpoint = tf.train.get_checkpoint_state("saved_networks")
#     if checkpoint and checkpoint.model_checkpoint_path:
#         saver.restore(sess, checkpoint.model_checkpoint_path)
#         print( "Successfully loaded:", checkpoint.model_checkpoint_path)
#     else:
#         print ("Could not find old network weights")

    epsilon = INITIAL_EPSILON
    t = 0
    while "pigs" != "fly":
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict = {s : [s_t]})[0]
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        for i in range(0, K):
            # run the selected action and observe next state and reward
            x_t1_col, r_t, terminal, bar1_score, bar2_score = game_state.frame_step(a_t)
            x_t1 = cv2.cvtColor(cv2.resize(x_t1_col, (80, 80)), cv2.COLOR_BGR2GRAY)
            ret, x_t1 = cv2.threshold(x_t1,1,255,cv2.THRESH_BINARY)
            x_t1 = np.reshape(x_t1, (80, 80, 1))
            s_t1 = np.append(x_t1, s_t[:,:,0:3], axis = 2)

            # store the transition in D
            D.append((s_t, a_t, r_t, s_t1, terminal))
            if len(D) > REPLAY_MEMORY:
                D.popleft()
        
        # only train if done observing
        if t > OBSERVE:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch})

        # update the old values
        s_t = s_t1
        t += 1

        # save progress every 10000 iterations
        if t % 10000 == 0:
            saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"
        #print ("TIMESTEP", t, "/ STATE", state, "/ LINES", game_state.total_lines, "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, "/ Q_MAX %e" % np.max(readout_t))
        if r_t!= 0:
            print ("TIMESTEP", t, "/ EPSILON", epsilon, "/ bar1_score", bar1_score, "/ bar2_score", bar2_score, "/ REWARD", r_t, "/ Q_MAX %e" % np.max(readout_t))
        
        
        win_score.pop(0)
        win_score.append(bar1_score - bar2_score)
        if(np.matrix(win_score).sum() > 72): #72
            print("Game_Ends_in Time:",int(time.time() - tick))
            break;   
        # write info to files
        '''
        if t % 10000 <= 100:
            a_file.write(",".join([str(x) for x in readout_t]) + '\n')
            h_file.write(",".join([str(x) for x in h_fc1.eval(feed_dict={s:[s_t]})[0]]) + '\n')
            cv2.imwrite("logs_tetris/frame" + str(t) + ".png", x_t1)
        '''

In [4]:
def playGame():
    sess = tf.InteractiveSession()
    s, readout, h_fc1 = createNetwork()
    trainNetwork(s, readout, h_fc1, sess)

In [ ]:
def main():
    playGame()

if __name__ == "__main__":
    tick = time.time()
    main()
    print("Game_Ends_in Time:",int(time.time() - tick))
    print("____________ END HERE _____________")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
TIMESTEP 297 / EPSILON 1.0 / bar1_score 0 / bar2_score 1 / REWARD -1 / Q_MAX 1.345254e-02
TIMESTEP 343 / EPSILON 1.0 / bar1_score 0 / bar2_score 2 / REWARD -1 / Q_MAX 1.512715e-02
TIMESTEP 389 / EPSILON 1.0 / bar1_score 0 / bar2_score 3 / REWARD -1 / Q_MAX 1.380204e-02
TIMESTEP 435 / EPSILON 1.0 / bar1_score 0 / bar2_score 4 / REWARD -1 / Q_MAX 1.366744e-02
TIMESTEP 481 / EPSILON 1.0 / bar1_score 0 / bar2_score 5 / REWARD -1 / Q_MAX 1.517603e-02
TIMESTEP 527 / EPSILON 0.9505999999999997 / bar1_score 0 / bar2_score 6 / REWARD -1 / Q_MAX 1.245254e-02
TIMESTEP 573 / EPSILON 0.8631999999999991 / bar1_score 0 / bar2_score 7 / REWARD -1 / Q_MAX 1.255603e-02
TIMESTEP 619 / EPSILON 0.7757999999999985 / bar1_score 0 / bar2_score 8 / REWARD -1 / Q_MAX 1.099862e-02
TIMESTEP 665 / EPSILON 0.6883999999999979 / bar1_score 0 / bar2_score 9 / REWARD -1 / Q_MAX 1.222987e-02
TIMESTEP 711 / EPSILON 0.6009999999999973 / bar1_score 0

TIMESTEP 5381 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 15 / REWARD -1 / Q_MAX -3.370636e-01
TIMESTEP 5427 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 16 / REWARD -1 / Q_MAX -3.542329e-01
TIMESTEP 5473 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 17 / REWARD -1 / Q_MAX -3.706146e-01
TIMESTEP 5519 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 18 / REWARD -1 / Q_MAX -3.527355e-01
TIMESTEP 5565 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 19 / REWARD -1 / Q_MAX -3.932435e-01
TIMESTEP 5778 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 0 / REWARD -1 / Q_MAX -3.893628e-01
TIMESTEP 5824 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 1 / REWARD -1 / Q_MAX -3.285970e-01
TIMESTEP 5870 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 2 / REWARD -1 / Q_MAX -3.454937e-01
TIMESTEP 5916 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 3 / REWARD -1 / Q_MAX -3.518159e-01
TIMESTEP 5962 / EPSILON

TIMESTEP 11226 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 11 / REWARD -1 / Q_MAX -3.589627e-01
TIMESTEP 11442 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 12 / REWARD -1 / Q_MAX -5.092101e-01
TIMESTEP 11488 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 13 / REWARD -1 / Q_MAX -6.181339e-01
TIMESTEP 11701 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 14 / REWARD -1 / Q_MAX -5.339075e-01
TIMESTEP 11747 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 15 / REWARD -1 / Q_MAX -3.327041e-01
TIMESTEP 11793 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 16 / REWARD -1 / Q_MAX -1.491739e-01
TIMESTEP 11839 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 17 / REWARD -1 / Q_MAX -6.432466e-01
TIMESTEP 11885 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 18 / REWARD -1 / Q_MAX -9.384148e-02
TIMESTEP 11931 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 19 / REWARD -1 / Q_MAX -6.739429e-01
TIMESTEP 1

TIMESTEP 15059 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 7 / REWARD -1 / Q_MAX -7.871455e-02
TIMESTEP 15105 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 8 / REWARD -1 / Q_MAX -8.472098e-01
TIMESTEP 15151 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 9 / REWARD -1 / Q_MAX -1.162039e-01
TIMESTEP 15197 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 10 / REWARD -1 / Q_MAX -9.269905e-02
TIMESTEP 15243 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 11 / REWARD -1 / Q_MAX -2.424206e-01
TIMESTEP 15289 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 12 / REWARD -1 / Q_MAX -7.315724e-02
TIMESTEP 15335 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 13 / REWARD -1 / Q_MAX -7.960382e-01
TIMESTEP 15381 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 14 / REWARD -1 / Q_MAX -8.188834e-01
TIMESTEP 15427 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 15 / REWARD -1 / Q_MAX -1.507670e-01
TIMESTEP 1547

TIMESTEP 18509 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 2 / REWARD -1 / Q_MAX -8.894081e-02
TIMESTEP 18555 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 3 / REWARD -1 / Q_MAX -3.572820e-01
TIMESTEP 18601 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 4 / REWARD -1 / Q_MAX -1.747822e-01
TIMESTEP 18647 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 5 / REWARD -1 / Q_MAX -3.779243e-01
TIMESTEP 18693 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 6 / REWARD -1 / Q_MAX -3.612857e-02
TIMESTEP 18739 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 7 / REWARD -1 / Q_MAX -4.385786e-01
TIMESTEP 18785 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 8 / REWARD -1 / Q_MAX -7.023561e-02
TIMESTEP 18831 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 9 / REWARD -1 / Q_MAX -4.305314e-01
TIMESTEP 18877 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 10 / REWARD -1 / Q_MAX -1.537549e-01
TIMESTEP 18923 / E

...

TIMESTEP 413746 / EPSILON 0.04999999999999416 / bar1_score 17 / bar2_score 18 / REWARD -1 / Q_MAX -4.769392e-01
TIMESTEP 413875 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 18 / REWARD 1 / Q_MAX 1.387835e+00
TIMESTEP 414004 / EPSILON 0.04999999999999416 / bar1_score 18 / bar2_score 19 / REWARD -1 / Q_MAX -1.468366e-01
TIMESTEP 414133 / EPSILON 0.04999999999999416 / bar1_score 19 / bar2_score 19 / REWARD 1 / Q_MAX 1.355433e+00
TIMESTEP 414516 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 0 / REWARD 1 / Q_MAX 1.307102e+00
TIMESTEP 414645 / EPSILON 0.04999999999999416 / bar1_score 0 / bar2_score 1 / REWARD -1 / Q_MAX -3.703489e-01
TIMESTEP 414774 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 1 / REWARD 1 / Q_MAX 1.366828e+00
TIMESTEP 414903 / EPSILON 0.04999999999999416 / bar1_score 1 / bar2_score 2 / REWARD -1 / Q_MAX -3.669227e-01
TIMESTEP 415032 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 2 / REWARD 1 / Q_MAX 1.274118e+00
TIMESTEP 415

TIMESTEP 427305 / EPSILON 0.04999999999999416 / bar1_score 2 / bar2_score 2 / REWARD 1 / Q_MAX 1.367661e+00
TIMESTEP 427686 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 2 / REWARD 1 / Q_MAX 1.669221e+00
TIMESTEP 427815 / EPSILON 0.04999999999999416 / bar1_score 3 / bar2_score 3 / REWARD -1 / Q_MAX -2.846228e-01
TIMESTEP 427945 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 3 / REWARD 1 / Q_MAX 1.316875e+00
TIMESTEP 428074 / EPSILON 0.04999999999999416 / bar1_score 4 / bar2_score 4 / REWARD -1 / Q_MAX -2.656498e-01
TIMESTEP 428203 / EPSILON 0.04999999999999416 / bar1_score 5 / bar2_score 4 / REWARD 1 / Q_MAX 1.317014e+00
TIMESTEP 428332 / EPSILON 0.04999999999999416 / bar1_score 5 / bar2_score 5 / REWARD -1 / Q_MAX -1.678786e-01
TIMESTEP 428461 / EPSILON 0.04999999999999416 / bar1_score 6 / bar2_score 5 / REWARD 1 / Q_MAX 1.313555e+00
TIMESTEP 428590 / EPSILON 0.04999999999999416 / bar1_score 6 / bar2_score 6 / REWARD -1 / Q_MAX -2.626106e-01
TIMESTEP 428719 / EP

TIMESTEP 441359 / EPSILON 0.04999999999999416 / bar1_score 7 / bar2_score 8 / REWARD -1 / Q_MAX -3.456656e-01
TIMESTEP 441488 / EPSILON 0.04999999999999416 / bar1_score 8 / bar2_score 8 / REWARD 1 / Q_MAX 1.199275e+00
TIMESTEP 441870 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 8 / REWARD 1 / Q_MAX 1.381623e+00
TIMESTEP 441999 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 9 / REWARD -1 / Q_MAX -2.972991e-01
TIMESTEP 442128 / EPSILON 0.04999999999999416 / bar1_score 10 / bar2_score 9 / REWARD 1 / Q_MAX 1.377016e+00
TIMESTEP 442509 / EPSILON 0.04999999999999416 / bar1_score 11 / bar2_score 9 / REWARD 1 / Q_MAX 1.469389e+00
TIMESTEP 442890 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 9 / REWARD 1 / Q_MAX 1.441695e+00
TIMESTEP 443019 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 10 / REWARD -1 / Q_MAX -4.572312e-01
TIMESTEP 443148 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 10 / REWARD 1 / Q_MAX 1.213898e+00
TIMESTEP 443529

TIMESTEP 463467 / EPSILON 0.04999999999999416 / bar1_score 7 / bar2_score 1 / REWARD -1 / Q_MAX -3.945559e-01
TIMESTEP 463596 / EPSILON 0.04999999999999416 / bar1_score 8 / bar2_score 1 / REWARD 1 / Q_MAX 1.242186e+00
TIMESTEP 463977 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 1 / REWARD 1 / Q_MAX 1.378326e+00
TIMESTEP 464359 / EPSILON 0.04999999999999416 / bar1_score 10 / bar2_score 1 / REWARD 1 / Q_MAX 1.549195e+00
TIMESTEP 464740 / EPSILON 0.04999999999999416 / bar1_score 11 / bar2_score 1 / REWARD 1 / Q_MAX 1.492506e+00
TIMESTEP 465122 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 1 / REWARD 1 / Q_MAX 1.538392e+00
TIMESTEP 465503 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 1 / REWARD 1 / Q_MAX 1.319966e+00
TIMESTEP 465884 / EPSILON 0.04999999999999416 / bar1_score 14 / bar2_score 1 / REWARD 1 / Q_MAX 1.480013e+00
TIMESTEP 466265 / EPSILON 0.04999999999999416 / bar1_score 15 / bar2_score 1 / REWARD 1 / Q_MAX 1.389971e+00
TIMESTEP 466646 / EP

TIMESTEP 487442 / EPSILON 0.04999999999999416 / bar1_score 6 / bar2_score 7 / REWARD -1 / Q_MAX 7.447886e-01
TIMESTEP 487571 / EPSILON 0.04999999999999416 / bar1_score 7 / bar2_score 7 / REWARD 1 / Q_MAX 1.425198e+00
TIMESTEP 487700 / EPSILON 0.04999999999999416 / bar1_score 7 / bar2_score 8 / REWARD -1 / Q_MAX -5.214808e-01
TIMESTEP 487832 / EPSILON 0.04999999999999416 / bar1_score 8 / bar2_score 8 / REWARD 1 / Q_MAX 1.069655e+00
TIMESTEP 488215 / EPSILON 0.04999999999999416 / bar1_score 9 / bar2_score 8 / REWARD 1 / Q_MAX 1.503901e+00
TIMESTEP 488596 / EPSILON 0.04999999999999416 / bar1_score 10 / bar2_score 8 / REWARD 1 / Q_MAX 1.476559e+00
TIMESTEP 488978 / EPSILON 0.04999999999999416 / bar1_score 11 / bar2_score 8 / REWARD 1 / Q_MAX 1.366789e+00
TIMESTEP 489359 / EPSILON 0.04999999999999416 / bar1_score 12 / bar2_score 8 / REWARD 1 / Q_MAX 1.335527e+00
TIMESTEP 489742 / EPSILON 0.04999999999999416 / bar1_score 13 / bar2_score 8 / REWARD 1 / Q_MAX 1.493922e+00
TIMESTEP 490123 / EPS

In [6]:
32265/60

537.75

In [3]:
print("Game_Ends_in Time:",int(time.time() - tick))